<a href="https://colab.research.google.com/github/ZefraAlseif/Project1_Wine_Network/blob/main/Project1_Wine_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adapting the original wine data to a regression model and classification model

In [1]:
#@title Data as Numpy Arrays
import matplotlib.pyplot as plt
import numpy as np
# This time we need to also import pandas
import pandas as pd
from io import StringIO

# Read in white wine data
# Uses PANDAS (pd) to create a PANDAS DataFrame Object:
white = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep = ';')

# Read in red wine data
# Uses PANDAS (pd) to create a PANDAS DataFrame Object:
red = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep =';')

red['type'] = 1
white['type'] = 0

wines = red.append(white, ignore_index = True)

# Import SKLEARN
import sklearn

# Import `train_test_split` from `sklearn.model_selection`
from sklearn.model_selection import train_test_split
# Specify the data -
X1 = wines.iloc[:, 0:11]
X2 = wines.iloc[:,12]
X = pd.concat([X1,X2],axis = 1)

y = np.ravel(wines.quality)

# Splitting the data set for training and validating - Done with SKLEARN
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.25, random_state = 45)

# Converting X_train & X_test DataFrame s to TF sensors
# Will use NumPy, TF, & Keras after this
# import tensorflow as tf

Xtrain = X_train.to_numpy()
X_valid = X_valid.to_numpy()

X_valid
# In reality:
# [1] ALL THE Xtrain patterns (with their y_train targets)
# will be used for TRAINING ([TR]), as Xtrain & y_train
# [2] MOST OF THE X_valid patterns (and their y_valid targets)
# will be used for VALIDATION ([TT]), as X_val & y_val
# BUT WE WILL SET ASIDE THE LAST 10 for "testing" ([TS])
# as X_tst & y_tst

# Retain the first 1615 patterns for validation ([TT])
Xval = X_valid[:1615]
Xval.shape

# and now set aside the last 10 for test
Xtst = X_valid[1615:]
Xtst.shape

# Same for the corresponding targets
# Retain the first 1615 for validation ([TT])
y_val = y_valid[:1615]
y_val.shape

y_tst = y_valid[1615:]
y_tst.shape 
y_tst

# Now, in addition, create the targets as one-hot-encoded 4 quality levels
# We will track these few targets through the conversion process
y_train[272:283]

# Function create rank-1 arrays where 3,4,5,6,7,8,9 are mapped to 1 or 2 or 3 or 4 
def to_4cs(x):
  lx = len(x)
  results = np.zeros(lx)
  for i in range(lx):
    # print("start")
    xa = x[i];
    if xa <= 3:
      results[i] = 1
    elif xa <= 6:
      results[i] = 2
    elif xa <= 8:
      results[i] = 3
    else:
      results[i] = 4
    # results [i, label] = 1
  results = results.astype(int)
  return results

train_labels = to_4cs(y_train)
val_labels = to_4cs(y_val)
tst_labels = to_4cs(y_tst)

# Let's verify that the training targets that we are tracking 
# were converted to levels (1 = BAD; 2 = Medium; 3 = GOOD; 4- Excellent) correctly:
train_labels[272:283]

# Now, one shot encoding of all 3 target arrays
# define a function to do the 

def to_one_hot(labels, dimension = 4):
  results = np.zeros((len(labels), dimension))
  for i, label in enumerate(labels-1):
    results[i, label] = 1.
  return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_val_labels = to_one_hot(val_labels)
one_hot_tst_labels = to_one_hot(tst_labels)

#Let's verify that the training targets we have tracked were
# one-hot encoded correctly
Xtrain.shape

# SO, AFTER EXECUTING THIS CELL, YOU WILL HAVE:
# FOR TRAINING:
# Xtrain (4872, 12)...y_train (4872,)...train_labels(4872,)....one_hot_train_labels (4872,4)
# FOR VALIDATING:
# Xval (1615, 12)...y_val (1615,)...val_labels(1615,)...one_hot_val_labels (1615,4)
# FOR TESTING:
# Xtst (10, 12)...y_tst (10,)...tst_labels(10,)... one_hot_tst_labels (10,4)
# PLEASE DO NOT CHANGE THE NAMES OF THESE VARIABLES (So that instructor can use them)


(4872, 12)

# Regression Model 1 (regmodl1)

In [ ]:
from keras.engine.input_layer import Input
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_regmodl1():
  regmodl1 = keras.Sequential(
      [
        layers.Dense(50, activation = 'relu'),
        layers.Dense(8, activation = 'relu'),
        layers.Dense(1, activation = 'relu')
      ]
)
  regmodl1.compile(optimizer = "rmsprop", loss = "mse", metrics = ["mae"])
  return regmodl1

regmodl1 = build_regmodl1()
regmodl1.fit(x = Xtrain,y = train_labels, batch_size = 32, epochs = 100, verbose = 2)

test_loss, test_mae = regmodl1.evaluate(Xval, val_labels)

regmodl1.summary()

# Regression Model 2 (regmodl2)

In [30]:
from keras.engine.input_layer import Input
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_regmodl2():
  regmodl2 = keras.Sequential(
      [
        layers.Dense(100, activation = 'relu'),
        layers.Dense(50, activation = 'relu'),
        layers.Dense(25, activation = 'relu'),
        layers.Dense(1, activation = 'relu')
      ]
  )
  regmodl2.compile(optimizer = "rmsprop", loss = "mse", metrics = ["mae"])
  return regmodl2

regmodl2 = build_regmodl2()
regmodl2.fit(x = Xtrain,y = train_labels, batch_size = 32, epochs = 100, verbose = 2)

test_loss, test_mae = regmodl2.evaluate(Xval, val_labels)

regmodl2.summary()

Epoch 1/100
153/153 - 1s - loss: 3.3148 - mae: 0.9715 - 906ms/epoch - 6ms/step
Epoch 2/100
153/153 - 0s - loss: 0.6636 - mae: 0.6267 - 243ms/epoch - 2ms/step
Epoch 3/100
153/153 - 0s - loss: 0.3991 - mae: 0.4861 - 233ms/epoch - 2ms/step
Epoch 4/100
153/153 - 0s - loss: 0.2793 - mae: 0.4149 - 215ms/epoch - 1ms/step
Epoch 5/100
153/153 - 0s - loss: 0.2382 - mae: 0.3736 - 243ms/epoch - 2ms/step
Epoch 6/100
153/153 - 0s - loss: 0.2057 - mae: 0.3487 - 238ms/epoch - 2ms/step
Epoch 7/100
153/153 - 0s - loss: 0.1893 - mae: 0.3322 - 222ms/epoch - 1ms/step
Epoch 8/100
153/153 - 0s - loss: 0.1794 - mae: 0.3241 - 217ms/epoch - 1ms/step
Epoch 9/100
153/153 - 0s - loss: 0.1746 - mae: 0.3162 - 235ms/epoch - 2ms/step
Epoch 10/100
153/153 - 0s - loss: 0.1696 - mae: 0.3094 - 242ms/epoch - 2ms/step
Epoch 11/100
153/153 - 0s - loss: 0.1656 - mae: 0.3066 - 220ms/epoch - 1ms/step
Epoch 12/100
153/153 - 0s - loss: 0.1605 - mae: 0.3045 - 225ms/epoch - 1ms/step
Epoch 13/100
153/153 - 0s - loss: 0.1572 - mae: 0

# Regression Model 3 (regmodl3)

In [ ]:
from keras.engine.input_layer import Input
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_regmodl2():
  regmodl2 = keras.Sequential(
      [
        layers.Dense(100, activation = 'relu'),
        layers.Dense(50, activation = 'relu'),
        layers.Dense(25, activation = 'relu'),
        layers.Dense(1, activation = 'relu')
      ]
  )
  regmodl2.compile(optimizer = "rmsprop", loss = "mse", metrics = ["mae"])
  return regmodl2

regmodl2 = build_regmodl2()
regmodl2.fit(x = Xtrain,y = train_labels, batch_size = 32, epochs = 100, verbose = 2)

test_loss, test_mae = regmodl2.evaluate(Xval, val_labels)

regmodl2.summary()

# Testing Networks

In [ ]:
network.predict(Xtrain[0:100])
